##### 20215129 김용수
## 대화를 장르로
#### 실행만 진행할 경우 맨 아래의 테스트 칸만 실행해주세요 (크롤링 없이 모델 불러와 실행)

# 크롤링을 통한 한글 노래 데이터 생성

#### 음악 스트리밍 사이트 "Bugs"에서 장르별 당일 TOP 100 노래들의 가사 링크를 크롤링 해옵니다.

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import time, random, os

if not os.path.exists("genre_lyrics.txt") and not (os.path.exists(os.getcwd() + "\\lyrics_korean_model.pt") or os.path.exists(os.getcwd() + "\\lyrics_english_model.pt")):
    song_pages = []
    genres = ["ballad", "dance", "folk", "idol", "rnh", "rns", "elec", "rock", "adultkpop"]

    for genre in genres:
        url = f"https://music.bugs.co.kr/genre/chart/kpop/{genre}/total/day"
        req = Request(url)
        req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36')
        html = urlopen(req)
        bsObj = BeautifulSoup(html, "html.parser")

        song_page_urls = []
        for cover in bsObj.body.find_all('a', {'class' : 'trackInfo'}):
            link = cover.get('href').split("?")[0]
            song_page_urls.append(link)

        time.sleep(random.uniform(1, 3))
        song_pages.append(song_page_urls)
        print(genre, "완료")
else:
    print("이미 크롤링을 한 기록이 있습니다. 다시 하기 위해서는 기존 크롤링 파일의 이름을 수정 또는 삭제해주세요.")

이미 크롤링을 한 기록이 있습니다. 다시 하기 위해서는 기존 크롤링 파일의 이름을 수정 또는 삭제해주세요.


#### 위의 각 노래 별 가사 링크에 접속하여 가사를 크롤링 해 하나의 배열로 만듭니다.
#### 성인 인증이 필요한 곡이거나 가사가 없는 곡의 경우 포함되지 않게 합니다.

In [2]:
if not os.path.exists("genre_lyrics.txt") and not (os.path.exists(os.getcwd() + "\\lyrics_korean_model.pt") or os.path.exists(os.getcwd() + "\\lyrics_english_model.pt")):
    genre_lyrics = []
    for genre_idx, song_urls in enumerate(song_pages):
        lyrics = []
        for idx, link in enumerate(song_urls):
            req2 = Request(link)
            req2.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36')
            html2 = urlopen(req2)
            bsObj2 = BeautifulSoup(html2, "html.parser")
            obj2 = bsObj2.find('div', {'class' : 'lyricsContainer'})
            if obj2.find("xmp") != None:
                lyric = obj2.select('xmp')[0].text
                lyrics.append([r for r in [result.strip() for result in lyric.split("\r\n" if "\r\n" in lyric else "\n")] if r != "" and r != "⠀"])
            if idx%10 == random.uniform(5, 7):
                time.sleep(random.uniform(1, 3))
            if idx%50 == 0:
                print(genres[genre_idx], ":", idx)
        genre_lyrics.append(lyrics)
        print(genres[genre_idx], "완료")
else:
    print("이미 크롤링을 한 기록이 있습니다. 다시 하기 위해서는 기존 크롤링 파일의 이름을 수정 또는 삭제해주세요.")

이미 크롤링을 한 기록이 있습니다. 다시 하기 위해서는 기존 크롤링 파일의 이름을 수정 또는 삭제해주세요.


#### 각 장르별 노래 개수를 보면 다른 장르들은 98이나 100으로 비슷하지만, elec 혼자 77인 것을 확인할 수 있습니다.
#### 이는 위에서 가사가 없는 곡의 경우 포함하지 않았기에 그렇습니다.

In [3]:
if not os.path.exists("genre_lyrics.txt") and not (os.path.exists(os.getcwd() + "\\lyrics_korean_model.pt") or os.path.exists(os.getcwd() + "\\lyrics_english_model.pt")):
    print(genre_lyrics[0][:3])
    for l in genre_lyrics:
        print(len(l))

#### 크롤링 해온 장르별 노래 가사들을 txt 파일로 저장하여 다음에도 불러와 사용 가능하게 합니다.

In [4]:
if not os.path.exists("genre_lyrics.txt") and not (os.path.exists(os.getcwd() + "\\lyrics_korean_model.pt") or os.path.exists(os.getcwd() + "\\lyrics_english_model.pt")):
    file_name = "genre_lyrics.txt"

    with open(file_name, "w", encoding="utf-8") as file:
        for genre_idx, lyrics in enumerate(genre_lyrics):
            for lyrics_idx, lyric in enumerate(lyrics):
                file.write("\n".join(map(str, lyric)) + "\n\n")
            file.write("Genre_End\n\n")
    print("done")
else:
    print("이미 크롤링을 한 기록이 있습니다. 다시 하기 위해서는 기존 크롤링 파일의 이름을 수정 또는 삭제해주세요.")

이미 크롤링을 한 기록이 있습니다. 다시 하기 위해서는 기존 크롤링 파일의 이름을 수정 또는 삭제해주세요.


# 한글 모델 생성

#### 위에서 크롤링한 가사들을 불러옵니다.

In [5]:
import torch, re, os
import numpy as np
from os import path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from datetime import datetime
from transformers import AutoTokenizer, AutoModel

np.random.seed(42)

In [6]:
file_name = "genre_lyrics.txt"

genres = ["ballad", "dance", "folk", "idol", "rnh", "rns", "elec", "rock", "adultkpop"]
genre_lyrics = []
current_lyrics = []

with open(file_name, "r", encoding="utf-8") as file:
    current_genre = []
    lyric = []
    pre = False
    for line in file:
        if line.strip() == "Genre_End":
            genre_lyrics.append(current_genre)
            current_genre = []
            pre = True
        elif line.strip() == "" and not pre:
            current_genre.append(lyric)
            lyric = []
        elif line.strip() == "":
            pre = False
        else:
            lyric.append(line.strip())

# 확인
# print(genre_lyrics)

#### 노래별 문장마다 나눠져 있던 가사를 합쳐 하나의 문장으로 만들어 불러옵니다.
#### 해당 과정 중 노래별 중복되는 문장을 하나로 합치고, 한글만을 뽑아 한 문장으로 합니다. 

In [7]:
lyrics_flatten = []
labels_flatten = []
for idx_g, g in enumerate(genre_lyrics):
    for idx_s, s in enumerate(g):
        lyrics_flatten.append(re.sub(f"[^가-힣| |]+", "", " ".join(set(s))))
        labels_flatten.append(genres[idx_g]) 
print(len(lyrics_flatten), len(labels_flatten))

873 873


In [8]:
lyrics_flatten[:2]

['만나지긴 할까 어떻게 서로를 기억해줄까 우릴 울리려 떠나간건 아냐 너도 같을거야 그때에 터트릴 웃음을 지금 질 수 있잖아 그걸로 충분해 서로 다른 그곳에서 고요를 거쳐 이제야 추억이 된 기억들 아직 타는 별 과거의 빛은 흐르고 그걸로 충분해 서로 다른 곳에서 몇 번의 사막을 거쳐 몇 번의 우기를 거쳐 잠시라도 우리 따뜻한 시간을 갖는다면 십년쯤 흘렀다고 그렇게 생각해봐 떠나간 모든 것은 시간따라 갔을 뿐 십년쯤 흘러가면 우린 어떻게 될까 가까스레 잠이 들다 애쓰던 잠은 떠났고',
 '새로 사귄 친구 함께 힘들단 걸 알지만 난생처음 준비한 선물 손을 잡고 늘 걷던 거리에 영영 다신 못 본다 해도 잠시라도 이 행복을 느껴서 고마웠다고 가까스로 본 너의 그 미소들 가보고 싶었던 식당 마주친 눈동자 바뀔까 봐 두려워 나는 사실 그대에게 좋은 사람이 아녜요 말할 수 없을 묘한 감정들이 고맙다는 너의 그 눈물들이 처음 본 네 얼굴 나는 사실 그대에게 좋은 사람이 되고 싶었어 이기적인 거 나도 잘 알아 그대 먼저 헤어지자 말해요 한 번은 널 볼 수 있을까 헤어지자고 말하려 오늘 첫눈을 보다가 문득 고백했던 그 순간 그댈 정말 사랑했다 말해요 그땐 그럴 수밖에 없던 그댈 위한 이 노래가 당신을 영원히 사랑할 테니 너에게 가다가 우리 추억 생각해 봤어 웃음 띤 네 얼굴 보면 시간이 지나고 나면 나는 어린 내게 한 번만 더 기회를 주길 어쩔 수 없을 걸 문득 너의 사진 보겠지 그대 이제 날 떠난다 말해요']

In [9]:
set(labels_flatten)

{'adultkpop', 'ballad', 'dance', 'elec', 'folk', 'idol', 'rnh', 'rns', 'rock'}

# 모델 학습

In [10]:
lyrics_train, lyrics_test, labels_train, labels_test = train_test_split(lyrics_flatten, labels_flatten, test_size=0.1, shuffle=True, random_state=34)

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


#### HuggingFace의 Transformer 라이브러리를 사용하였습니다. 
#### 토크나이저는 pre-train 된 beomi/kcbert-base를 불러와 사용했고,
#### 장르를 분류하기 위해 BertForSequenceClassfication으로 beomi/kcbert-base를 불러와 모델을 생성하였습니다.

In [12]:
# pre = "skt/kobert-base-v1"
pre = "beomi/kcbert-base"

# pre-trained word embedding을 불러옵니다.
tokenizer_korean = AutoTokenizer.from_pretrained(pre)

# 레이블 값들을 숫자로 변환합니다.
label_dict_korean = {label: idx for idx, label in enumerate(genres)}
label_indices_korean = [label_dict_korean[label] for label in labels_train]

# 가사를 토큰화 하면서 모델에 맞는 입력 형식으로 변환합니다.
encoded_lyrics = tokenizer_korean.batch_encode_plus(
    lyrics_train,
    add_special_tokens=True,
    padding='longest',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
).to(device)

# 레이블 값들의 형식을 Tensor로 변환합니다.
labels = torch.tensor(label_indices_korean)

# 기존 학습한 모델이 있다면 해당 모델을, 아니라면 위에서 지정한 BERT 모델을 불러옵니다.
model_path = "lyrics_korean_model.pt"
if path.exists(os.getcwd() + "\\" + model_path):
    model_korean = BertForSequenceClassification.from_pretrained("./" + model_path)
    model_korean.to(device)
    print("사전 학습된 모델 불러오기 성공")
else:
    model_korean = BertForSequenceClassification.from_pretrained(pre, num_labels=len(label_dict_korean))
    model_korean.to(device)

    # 훈련 모드 설정
    model_korean.train()

    # 옵티마이저 설정
    optimizer = torch.optim.AdamW(model_korean.parameters(), lr=1e-5)

    # 훈련
    epochs = 20
    batch_size = 16
    
    loss_arr = []

    for epoch in range(epochs):
        total_loss = 0
        print("epochs:", epoch+1, "시작")
        for i in range(0, len(encoded_lyrics['input_ids']), batch_size):
            input_batch = encoded_lyrics['input_ids'][i:i+batch_size].to(device)
            label_batch = labels[i:i+batch_size].to(device)
            attention_mask = encoded_lyrics['attention_mask'][i:i+batch_size]

            optimizer.zero_grad()

            outputs = model_korean(input_batch, labels=label_batch, token_type_ids=None, attention_mask=attention_mask)

            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_korean.parameters(), 1.0)
            optimizer.step()

        avg_loss = total_loss / len(encoded_lyrics['input_ids'])
        loss_arr.append(avg_loss)
        print(f'Epoch {epoch + 1} - Loss: {avg_loss}', datetime.now().strftime("%H:%M:%S"))
    plt.plot(loss_arr)
    plt.show()

    # 모델 저장
    model_korean.save_pretrained(model_path)

사전 학습된 모델 불러오기 성공


In [13]:
accur_sum = 0
for lyric, label in zip(lyrics_test, labels_test):
    # 테스트 가사를 토큰화 하면서 모델에 맞는 입력 형식으로 변환합니다.
    test_input_ids =  torch.tensor(tokenizer_korean.batch_encode_plus([lyric])['input_ids']).to(device)

    # 평가 모드 설정
    model_korean.eval()

    # 예측
    with torch.no_grad():
        output = model_korean(test_input_ids[:, :300])

    # 결과의 최대값으로 인덱스 예측
    predicted_genre_index = torch.argmax(output['logits'], dim=1).item()
    predicted_genre = {idx: genre for genre, idx in label_dict_korean.items()}[predicted_genre_index]
    if predicted_genre == label:
        accur_sum += 1
print("정확도 {:.1f}%".format((accur_sum/len(lyrics_test))*100))

Token indices sequence length is longer than the specified maximum sequence length for this model (340 > 300). Running this sequence through the model will result in indexing errors


정확도 39.8%


In [14]:
if device == torch.device('cuda'):
    torch.cuda.empty_cache()

# 영어 모델 생성

In [15]:
import torch, re, os
import pandas as pd
import numpy as np
from os import path
import matplotlib.pyplot as plt
from datetime import datetime
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertForSequenceClassification

np.random.seed(42)

#### 영어 가사 데이터를 불러옵니다.

In [16]:
# https://huggingface.co/datasets/Veucci/lyric-to-3genre
train_dataset = pd.read_csv("data_3genre.csv")
train_dataset

0|I walked through the door with you the air was cold But somethin bout it felt like home somehow And I left my scarf there at your sisters house And youve still got it in your drawer even now Oh your sweet disposition and my wideeyed gaze Were singin in the car getting lost upstate Autumn leaves fallin down like pieces into place And I can picture it after all these days And I know its long gone and That magics not here no more And I might be okay but Im not fine at all Oh oh oh Causе there we arе again on that little town street You almost ran the red cause you were lookin over at me Wind in my hair I was there I remember it all too well Photo album on the counter your cheeks were turnin red You used to be a little kid with glasses in a twinsized bed And your mothers tellin stories bout you on the teeball team You taught me bout your past thinkin your future was me And you were tossing me the car keys Fuck the patriarchy Keychain on the ground we were always skippin town And I was thinkin on the drive down Any time now Hes gonna say its love you never called it what it was Til we were dead and gone and buried Check the pulse and come back swearin its the same After three months in the grave And then you wondered where it went to as I reached for you But all I felt was shame and you held my lifeless frame You might also like  And I know its long gone and There was nothing else I could do And I forget about you long enough To forget why I needed to Cause there we are again in the middle of the night Were dancin round the kitchen in the refrigerator light Down the stairs I was there I remember it all too well And there we are again when nobody had to know You kept me like a secret but I kept you like an oath Sacred prayer and wed swear To remember it all too well yeah Well maybe we got lost in translation maybe I asked for too much But maybe this thing was a masterpiece til you tore it all up Runnin scared I was there I remember it all too well And you call me up again just to break me like a promise So casually cruel in the name of bein honest Im a crumpledup piece of paper lyin here Cause I remember it all all all They say alls well that ends well but Im in a new hell Every time you doublecross my mind You said if we had been closer in age maybe it wouldve been fine And that made me want to die The idea you had of me who was she A neverneedy everlovely jewel whose shine reflects on you Not weepin in a party bathroom Some actress askin me what happened you Thats what happened you You who charmed my dad with selfeffacing jokes Sippin coffee like youre on a latenight show But then he watched me watch the front door all night willin you to come And he said Its supposed to be fun turning twentyone Time wont fly its like Im paralyzed by it Id like to be my old self again but Im still tryin to find it After plaid shirt days and nights when you made me your own Now you mail back my things and I walk home alone But you keep my old scarf from that very first week Cause it reminds you of innocence and it smells like me You cant get rid of it Cause you remember it all too well yeah Cause there we are again when I loved you so Back before you lost the one real thing youve ever known It was rare I was there I remember it all too well Wind in my hair you were there You remember it all Down the stairs you were there You remember it all It was rare I was there I remember it all too well And I was never good at tellin jokes but the punch line goes Ill get older but your lovers stay my age From when your Brooklyn broke my skin and bones Im a soldier whos returning half her weight And did the twin flame bruise paint you blue Just between us did the love affair maim you too Cause in this citys barren cold I still remember the first fall of snow And how it glistened as it fell I remember it all too well Just between us did the love affair maim you all too well Just between us do you remember it all too well Just between us I remember it (Just betw

#### 데이터를 레이블과 가사로 나누고 가사의 불필요한 요소를 제거합니다.

In [17]:
lyrics_flatten = []
labels_flatten = []
genres = ["pop", "rock", "hip-hop"]

for i in range(len(train_dataset)):
    label, lyrics = str(train_dataset.iloc[i].values[0]).split("|")
    labels_flatten.append(label)
    lyrics_flatten.append(re.sub(r'\d+$', '', re.sub(r"(\u00A0|\u1680|\u180E|\u2005|\u200B|\u202F|\u205F|\u3000|\uFEFF)", " ", lyrics)))
    
print(len(lyrics_flatten), len(labels_flatten))

3013 3013


In [18]:
set(labels_flatten)

{'0', '1', '2'}

# 모델 학습

In [19]:
lyrics_train, lyrics_test, labels_train, labels_test = train_test_split(lyrics_flatten, labels_flatten, test_size=0.1, shuffle=True, random_state=34)

In [20]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [21]:
pre = "bert-base-cased"

# pre-trained word embedding을 불러옵니다.
tokenizer_english = BertTokenizer.from_pretrained(pre)

# 레이블 값들을 숫자로 변환합니다.
label_dict_english = {label: idx for idx, label in enumerate(['0', '1', '2'])}
label_indices_english = [label_dict_english[label] for label in labels_train]

# 가사를 토큰화 하면서 모델에 맞는 입력 형식으로 변환합니다.
encoded_lyrics = tokenizer_english.batch_encode_plus(
    lyrics_train,
    add_special_tokens=True,
    padding='longest',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
).to(device)

# 레이블 값들의 형식을 Tensor로 변환합니다.
labels = torch.tensor(label_indices_english)

# 기존 학습한 모델이 있다면 해당 모델을, 아니라면 위에서 지정한 BERT 모델을 불러옵니다.
model_path = "lyrics_english_model.pt"
if path.exists(os.getcwd() + "\\" + model_path):
    model_english = BertForSequenceClassification.from_pretrained("./" + model_path, num_labels=len(label_dict_english))
    model_english.to(device)
    print("사전 학습된 모델 불러오기 성공")
else:
    model_english = BertForSequenceClassification.from_pretrained(pre, num_labels=len(label_dict_english))
    model_english.to(device)

    # 훈련 모드 설정
    model_english.train()

    # 옵티마이저 설정
    optimizer = torch.optim.AdamW(model_english.parameters(), lr=2e-5)

    # 훈련
    epochs = 24
    batch_size = 8

    loss_arr = []
    
    for epoch in range(epochs):
        total_loss = 0
        print("epochs:", epoch+1, "시작")
        for i in range(0, len(encoded_lyrics['input_ids']), batch_size):
            input_batch = encoded_lyrics['input_ids'][i:i+batch_size].to(device)
            label_batch = labels[i:i+batch_size].to(device)
            attention_mask = encoded_lyrics['attention_mask'][i:i+batch_size]

            optimizer.zero_grad()

            outputs = model_english(input_batch, labels=label_batch, token_type_ids=None, attention_mask=attention_mask)

            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_english.parameters(), 1.0)
            optimizer.step()

        avg_loss = total_loss / len(encoded_lyrics['input_ids'])
        loss_arr.append(avg_loss)
        print(f'Epoch {epoch + 1} - Loss: {avg_loss}', datetime.now().strftime("%H:%M:%S"))
    plt.plot(loss_arr)
    plt.show()

    # 모델 저장
    model_english.save_pretrained(model_path)

사전 학습된 모델 불러오기 성공


In [22]:
accur_sum = 0
for lyric, label in zip(lyrics_test, labels_test):
    # 테스트 가사를 토큰화 하면서 모델에 맞는 입력 형식으로 변환합니다.
    test_input_ids =  torch.tensor(tokenizer_english.batch_encode_plus([lyric])['input_ids']).to(device)

    # 평가 모드 설정
    model_english.eval()

    # 예측
    with torch.no_grad():
        output = model_english(test_input_ids[:, :512])

    # 결과의 최대값으로 인덱스 예측
    predicted_genre_english_index = torch.argmax(output['logits'], dim=1).item()
    predicted_genre_english = {idx: genre for genre, idx in label_dict_english.items()}[predicted_genre_english_index]
    if predicted_genre_english == label:
        accur_sum += 1
print("정확도 {:.1f}%".format((accur_sum/len(lyrics_test))*100))

Token indices sequence length is longer than the specified maximum sequence length for this model (1136 > 512). Running this sequence through the model will result in indexing errors


정확도 86.1%


# 테스트 입력 (자유롭게 입력해보세요)
#### 이 부분만 실행하시면 됩니다.

In [2]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModel
np.random.seed(42)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# 모델 불러오기
tokenizer_korean = AutoTokenizer.from_pretrained("beomi/kcbert-base")
tokenizer_english = BertTokenizer.from_pretrained("bert-base-cased")

model_korean = BertForSequenceClassification.from_pretrained("./lyrics_korean_model.pt")
model_korean.to(device)

model_english = BertForSequenceClassification.from_pretrained("./lyrics_english_model.pt")
model_english.to(device)

# 모델 예측 결과 테스트
genres_model_korean = ["ballad", "dance", "folk", "idol", "rnh", "rns", "elec", "rock", "adultkpop"]
genres_english = ["팝", "락", "힙합"]
genres_korean = ["발라드", "댄스/팝", "포크/어쿠스틱", "아이돌", "랩/힙합", "알앤비/소울", "일렉트로닉", "락/메탈", "성인가요"]

while True:
    # 사용자로부터 문장을 입력 받아옴
    print("\n입력")
    text=input()
    if text == "--end":
        break

    # 입력된 문장을 모델에 맞는 입력 형식으로 변환하고 Tensor로 변환
    test_input_ids_korean =  torch.tensor(tokenizer_korean.batch_encode_plus([text])['input_ids']).to(device)
    test_input_ids_english =  torch.tensor(tokenizer_english.batch_encode_plus([text])['input_ids']).to(device)
    
    # 평가 모드 설정
    model_korean.eval()
    model_english.eval()

    # 예측
    with torch.no_grad():
        output_korean = model_korean(test_input_ids_korean[:, :300])
        output_english = model_english(test_input_ids_english[:, :300])
    
    # 결과의 최대값으로 인덱스 예측
    predicted_genre_korean_index = torch.argmax(output_korean['logits'], dim=1).item()
    predicted_genre_korean = genres_korean[predicted_genre_korean_index]
    predicted_genre_english_index = torch.argmax(output_english['logits'], dim=1).item()
    predicted_genre_english = genres_english[predicted_genre_english_index]
    print("한국어 모델 | 입력한 문장의 예측된 장르:", predicted_genre_korean)
    print("  영어 모델 | 입력한 문장의 예측된 장르:", predicted_genre_english)


입력


 밤은 다시 길고 깊어졌네 나는 점점 너로 잠 못 들게 돼 글로 적어내긴 어려운 이 기분을 너도 느꼈으면 좋겠는데 너는 아무 생각 없이 몇 번 나를 지나가며 웃은 거라지만 나의 하얀 옷에 너의 잉크가 묻어 닦아낼 수 없을 만큼 번졌네


한국어 모델 | 입력한 문장의 예측된 장르: 포크/어쿠스틱
  영어 모델 | 입력한 문장의 예측된 장르: 락

입력


 --end


## 봐주셔서 감사합니다